In [ ]:
import os
import shutil
import sys

# 1. CLEANUP (Wipe previous failed attempts)
if os.path.exists("VideoLLaMA2"):
    print("🧹 Cleaning up old files...")
    shutil.rmtree("VideoLLaMA2")

# 2. CLONE
print("⬇️ Cloning repository...")
!git clone https://github.com/DAMO-NLP-SG/VideoLLaMA2.git
%cd VideoLLaMA2
!git checkout audio_visual

# 3. THE FIX: Remove ALL version constraints
# We tell the AI: "Use whatever versions Colab already has."
print("🛠️ Patching for Colab 2025...")

# Remove Numpy & Torch constraints entirely (Fixes the binary crash)
!sed -i '/numpy/d' pyproject.toml
!sed -i '/numpy/d' requirements.txt
!sed -i '/torch/d' requirements.txt
!sed -i '/torchvision/d' requirements.txt

# Remove Flash Attention (Incompatible with T4)
!sed -i '/flash-attn/d' pyproject.toml
!sed -i '/flash-attn/d' requirements.txt

# 4. COMPATIBILITY PATCH (Monkey Patch)
# Numpy 2.0 removed 'np.float'. We add it back so old code doesn't break.
import numpy as np
if not hasattr(np, 'float'):
    np.float = float
    print("🩹 Applied Numpy 2.0 Patch")

# 5. INSTALL
print("📦 Installing dependencies (Keep existing versions)...")
# We use --no-deps for the library itself to prevent it from downgrading anything
!pip install -r requirements.txt --quiet
!pip install ffmpeg-python bitsandbytes accelerate deepspeed --quiet
!pip install -e . --no-build-isolation --no-deps

# 6. VERIFY
%cd ..
sys.path.append("/content/VideoLLaMA2")
try:
    from videollama2 import model_init, mm_infer
    print("\n✅ SUCCESS: Setup Verified. The crash is fixed.")
except ImportError as e:
    print(f"\n❌ ERROR: {e}")
except ValueError as e:
    print(f"\n❌ VERSION ERROR: {e}")
    print("Did you forget to 'Restart Session' before running this?")

🧹 Cleaning up old files...
⬇️ Cloning repository...
Cloning into 'VideoLLaMA2'...
remote: Enumerating objects: 986, done.
remote: Counting objects: 100% (427/427), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 986 (delta 361), reused 245 (delta 245), pack-reused 559 (from 2)
Receiving objects: 100% (986/986), 55.81 MiB | 39.77 MiB/s, done.
Resolving deltas: 100% (661/661), done.
/content/VideoLLaMA2
Branch 'audio_visual' set up to track remote branch 'audio_visual' from 'origin'.
Switched to a new branch 'audio_visual'
🛠️ Patching for Colab 2025...
🩹 Applied Numpy 2.0 Patch
📦 Installing dependencies (Keep existing versions)...
Obtaining file:///content/VideoLLaMA2
  Checking if build backend supports build_editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for videollama2 (pyproject.toml) ... done
  Created wheel for videollama2: filename=videollama2-1.0-0.editable-py3-none-any.whl size=13152 sha256=be3e9cf5da6c5600199

/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:54: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]



✅ SUCCESS: Setup Verified. The crash is fixed.


In [ ]:
import torch
import gc
from videollama2 import model_init, mm_infer
from videollama2.utils import disable_torch_init

# 1. Clear RAM before loading
gc.collect()
torch.cuda.empty_cache()

# 2. Define Path
model_path = "DAMO-NLP-SG/VideoLLaMA2.1-7B-AV"
print("⏳ Downloading Model (4-bit Mode)...")

disable_torch_init()

# 3. Load Model
# I removed 'low_cpu_mem_usage=True' because the library adds it automatically.
model, processor, tokenizer = model_init(
    model_path,
    load_in_4bit=True,  # This keeps it small (approx 5GB)
    device_map="auto"
)

print("✅ Model Loaded & Ready!")

⏳ Downloading Model (Ultra-Light 4-bit Mode)...


Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


TypeError: transformers.modeling_utils.PreTrainedModel.from_pretrained() got multiple values for keyword argument 'low_cpu_mem_usage'